### Training the model on category choosen

In [1]:
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
import wandb
from datasets import load_from_disk, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, \
                         DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

d:\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Ashwin
[nltk_data]     Prakash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# wandb.init(project="abstract-to-title", entity="nerdimite")

#### Preprocess Data

In [3]:
# Initialize T5-base tokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [4]:
# Load the processed data
dataset = load_from_disk('arxiv_AI_dataset')

In [5]:
MAX_SOURCE_LEN = 512
MAX_TARGET_LEN = 128

In [6]:
def preprocess_data(example):
    
    model_inputs = tokenizer(example['abstract'], max_length=MAX_SOURCE_LEN, padding=True, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['title'], max_length=MAX_TARGET_LEN, padding=True, truncation=True)

    # Replace all pad token ids in the labels by -100 to ignore padding in the loss
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs['labels'] = labels["input_ids"]

    return model_inputs

In [7]:
# Apply preprocess_data() to the whole dataset
processed_dataset = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=['abstract', 'title'],
    desc="Running tokenizer on dataset",
)

processed_dataset

Parameter 'function'=<function preprocess_data at 0x000001E3ED7DA438> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Running tokenizer on dataset: 100%|██████████| 3/3 [00:02<00:00,  1.01ba/s]


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 46972
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 2610
    })
    val: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 2610
    })
})

#### Training Parameters

In [8]:
batch_size = 8
num_epochs = 5
learning_rate = 5.6e-5
weight_decay = 0.01
log_every = 50
eval_every = 1000
lr_scheduler_type = "linear"

In [9]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="model-t5-base",
    evaluation_strategy="steps",
    eval_steps=eval_every,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_steps=500,
    save_total_limit=3,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    logging_steps=log_every,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb",
    resume_from_checkpoint=True,
)

#### Train

In [10]:
# Initialize T5-base model
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

In [11]:
# Define ROGUE metrics on evaluation data
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Compute ROUGE scores and get the median scores
    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    return {k: round(v, 4) for k, v in result.items()}

In [12]:
# Dynamic padding in batch using a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [13]:
# Define the trainer
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.


In [ ]:
trainer.train()

#### Evaluation

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained('model')
tokenizer = AutoTokenizer.from_pretrained('model')

#### Prediction

In [3]:
temperature = 0.9
num_beams = 4
max_gen_length = 128

In [21]:
abstract = """In this paper, we question if self-supervised learning provides
new properties to Vision Transformer (ViT) [19] that
stand out compared to convolutional networks (convnets).
Beyond the fact that adapting self-supervised methods to this
architecture works particularly well, we make the following
observations: first, self-supervised ViT features contain
explicit information about the semantic segmentation of an
image, which does not emerge as clearly with supervised
ViTs, nor with convnets. Second, these features are also excellent
k-NN classifiers, reaching 78.3% top-1 on ImageNet
with a small ViT. Our study also underlines the importance of
momentum encoder [33], multi-crop training [10], and the
use of small patches with ViTs. We implement our findings
into a simple self-supervised method, called DINO, which
we interpret as a form of self-distillation with no labels.
We show the synergy between DINO and ViTs by achieving
80.1% top-1 on ImageNet in linear evaluation with ViT-Base"""
# abstract = dataset['test'][0]['abstract']
inputs = tokenizer([abstract], max_length=512, return_tensors='pt')

title_ids = model.generate(
    inputs['input_ids'], 
    num_beams=num_beams, 
    temperature=temperature, 
    max_length=max_gen_length, 
    early_stopping=True
)
title = tokenizer.decode(title_ids[0].tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(title)

Self-Supervised Learning of Vision Transformers


In [4]:
import gradio as gr

def inputExample(text):
    temperature = 0.9
    num_beams = 4
    max_gen_length = 128
    inputs = tokenizer([text], max_length=512, return_tensors='pt')

    title_ids = model.generate(
        inputs['input_ids'], 
        num_beams=num_beams, 
        temperature=temperature, 
        max_length=max_gen_length, 
        early_stopping=True
    )
    title = tokenizer.decode(title_ids[0].tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return title

demo = gr.Interface(
    fn=inputExample,
    inputs=gr.inputs.Textbox(lines=5, label="Input Text"),
    outputs=gr.outputs.Textbox(label="Generated Text"),
)
demo.launch()

d:\Anaconda3\envs\nlp\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
d:\Anaconda3\envs\nlp\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
d:\Anaconda3\envs\nlp\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
d:\Anaconda3\envs\nlp\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
d:\Anaconda3\envs\nlp\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please im

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x1af1f692288>, 'http://127.0.0.1:7860/', None)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
